This notebook is for the Capstone project

In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [3]:
#Import additional packages
from sklearn.preprocessing import StandardScaler
import time
import seaborn as sns
from tqdm.auto import tqdm, trange

!pip install pyscreenshot

#Set up parallel processing
import concurrent.futures
import multiprocessing
num_processes = multiprocessing.cpu_count()
print("There are "+str(num_processes)+" logical cores on this system.")
global_starttime = time.time()

     |████████████████████████████████| 81kB 5.6MB/s eta 0:00:011
There are 56 logical cores on this system.


# Data acquisition

This project aims to provide insight in the the factors that influence the occurance and the severity of road traffic accidents specifically in the Seattle area. For this data off road traffic accidents between 2004-2020 was used. This dataset was obtained from the Seattle Open Data portal here:http://data-seattlecitygis.opendata.arcgis.com/datasets/5b5c745e0f1f48e7a53acec63a0022ab_0
A PDF with additional information is available as well, in this details of the attribute metadata can be obtained.

In [6]:
!wget -O Collisions.csv https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv


--2020-10-13 15:02:18--  https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv
Resolving s3.us.cloud-object-storage.appdomain.cloud (s3.us.cloud-object-storage.appdomain.cloud)... 67.228.254.196
Connecting to s3.us.cloud-object-storage.appdomain.cloud (s3.us.cloud-object-storage.appdomain.cloud)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73917638 (70M) [text/csv]
Saving to: ‘Collisions.csv’

100%[======================================>] 73,917,638  30.9MB/s   in 2.3s   

2020-10-13 15:02:21 (30.9 MB/s) - ‘Collisions.csv’ saved [73917638/73917638]



In [7]:
#load data
!wget -O Collisions.csv https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv

#Read data from the downloaded portal dataset and import to a dataframe
df = pd.read_csv("Collisions.csv")

df_shape_orig = df.shape
print("Shape of the original data frame: "+str(df.shape[0])+' x '+str(df.shape[1]))

/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Shape of the original data frame: 194673 x 38


In [10]:
#Analyse the intial load
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df.head(25)

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,EXCEPTRSNCODE,EXCEPTRSNDESC,SEVERITYCODE.1,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,5TH AVE NE AND NE 103RD ST,,NaN,2,Injury Collision,Angles,2,0,0,2,2013/03/27 00:00:00+00,3/27/2013 2:54:00 PM,At Intersection (intersection related),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Overcast,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,AURORA BR BETWEEN RAYE ST AND BRIDGE WAY N,NaN,NaN,1,Property Damage Only Collision,Sideswipe,2,0,0,2,2006/12/20 00:00:00+00,12/20/2006 6:55:00 PM,Mid-Block (not related to intersection),16,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE ...",NaN,0,Raining,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,4TH AVE BETWEEN SENECA ST AND UNIVERSITY ST,NaN,NaN,1,Property Damage Only Collision,Parked Car,4,0,0,3,2004/11/18 00:00:00+00,11/18/2004 10:20:00 AM,Mid-Block (not related to intersection),14,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",NaN,0,Overcast,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,2ND AVE BETWEEN MARION ST AND MADISON ST,,NaN,1,Property Damage Only Collision,Other,3,0,0,3,2013/03/29 00:00:00+00,3/29/2013 9:26:00 AM,Mid-Block (not related to intersection),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Clear,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,SWIFT AVE S AND SWIFT AV OFF RP,NaN,NaN,2,Injury Collision,Angles,2,0,0,2,2004/01/28 00:00:00+00,1/28/2004 8:04:00 AM,At Intersection (intersection related),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,0,Raining,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N
5,1,-122.387598,47.690575,6,320840,322340,E919477,Matched,Intersection,36974.0,24TH AVE NW AND NW 85TH ST,,NaN,1,Property Damage Only Collision,Angles,2,0,0,2,2019/04/20 00:00:00+00,4/20/2019 5:42:00 PM,At Intersection (intersection related),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Clear,Dry,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
6,1,-122.338485,47.618534,7,83300,83300,3282542,Matched,Intersection,29510.0,DENNY WAY AND WESTLAKE AVE,NaN,NaN,1,Property Damage Only Collision,Angles,2,0,0,2,2008/12/09 00:00:00+00,12/9/2008,At Intersection (intersection related),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,0,Raining,Wet,Daylight,NaN,8344002.0,NaN,10,Entering at angle,0,0,N
7,2,-122.320780,47.614076,9,330897,332397,EA30304,Matched,Intersection,29745.0,BROADWAY AND E PIKE ST,,NaN,2,Injury Collision,Cycles,3,0,1,1,2020/04/15 00:00:00+00,4/15/2020 5:47:00 PM,At Intersection (intersection related),51,PEDALCYCLIST STRUCK MOTOR VEHICLE FRONT END AT...,NaN,N,Clear,Dry,Daylight,NaN,NaN,NaN,5,Vehicle Strikes Pedalcyclist,6855,0,N
8,1,-122.335930,47.611904,10,63400,63400,2071243,Matched,Block,NaN,PINE ST BETWEEN 5TH AVE AND 6TH AVE,NaN,NaN,1,Property Damage Only Collision,Parked Car,2,0,0,2,2006/06/15 00:00:00+00,6/15/2006 1:00:00 PM,Mid-Block (not related to intersection),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,0,Clear,Dry,Daylight,NaN,6166014.0,NaN,32,One parked--one moving,0,0,N
9,2,-122.384700,47.528475,12,58600,58600,2072105,Matched,Intersection,34679.0,41ST AVE SW AND SW THISTLE ST,NaN,NaN,2,Injury Collision,Angles,2,0,0,2,2006/03/20 00:00:00+00,3/20/2006 3:49:00 PM,At Intersection (intersection related),11,"MOTOR VEHICLE STRUCK M

In [11]:
#Verify the data types
df.dtypes

SEVERITYCODE        int64
X                 float64
Y                 float64
OBJECTID            int64
INCKEY              int64
COLDETKEY           int64
REPORTNO           object
STATUS             object
ADDRTYPE           object
INTKEY            float64
LOCATION           object
EXCEPTRSNCODE      object
EXCEPTRSNDESC      object
SEVERITYCODE.1      int64
SEVERITYDESC       object
COLLISIONTYPE      object
PERSONCOUNT         int64
PEDCOUNT            int64
PEDCYLCOUNT         int64
VEHCOUNT            int64
INCDATE            object
INCDTTM            object
JUNCTIONTYPE       object
SDOT_COLCODE        int64
SDOT_COLDESC       object
INATTENTIONIND     object
UNDERINFL          object
WEATHER            object
ROADCOND           object
LIGHTCOND          object
PEDROWNOTGRNT      object
SDOTCOLNUM        float64
SPEEDING           object
ST_COLCODE         object
ST_COLDESC         object
SEGLANEKEY          int64
CROSSWALKKEY        int64
HITPARKEDCAR       object
dtype: objec

In [12]:
print(df.loc[0,:])

SEVERITYCODE                                                      2
X                                                          -122.323
Y                                                           47.7031
OBJECTID                                                          1
INCKEY                                                         1307
COLDETKEY                                                      1307
REPORTNO                                                    3502005
STATUS                                                      Matched
ADDRTYPE                                               Intersection
INTKEY                                                        37475
LOCATION                                 5TH AVE NE AND NE 103RD ST
EXCEPTRSNCODE                                                      
EXCEPTRSNDESC                                                   NaN
SEVERITYCODE.1                                                    2
SEVERITYDESC                                    

In [13]:
#Check where there are NaNs in the dataframe
print(df.isnull().sum(axis=0))

SEVERITYCODE           0
X                   5334
Y                   5334
OBJECTID               0
INCKEY                 0
COLDETKEY              0
REPORTNO               0
STATUS                 0
ADDRTYPE            1926
INTKEY            129603
LOCATION            2677
EXCEPTRSNCODE     109862
EXCEPTRSNDESC     189035
SEVERITYCODE.1         0
SEVERITYDESC           0
COLLISIONTYPE       4904
PERSONCOUNT            0
PEDCOUNT               0
PEDCYLCOUNT            0
VEHCOUNT               0
INCDATE                0
INCDTTM                0
JUNCTIONTYPE        6329
SDOT_COLCODE           0
SDOT_COLDESC           0
INATTENTIONIND    164868
UNDERINFL           4884
WEATHER             5081
ROADCOND            5012
LIGHTCOND           5170
PEDROWNOTGRNT     190006
SDOTCOLNUM         79737
SPEEDING          185340
ST_COLCODE            18
ST_COLDESC          4904
SEGLANEKEY             0
CROSSWALKKEY           0
HITPARKEDCAR           0
dtype: int64


# Data Cleaning

The data obtained from the Seattle Open Data website are not, in their original form, usable for the purposes of model building. There are a number of issues which must be addressed:

Rows which are missing information about the target variable: the objective of this model is to predict the car accident severity code ( SEVERITYCODE ) from the given features. However as we can see from the previous cell, there are 21,602 accidents in the dataframe with SEVERITYCODE =0, corresponding to an accident with "Unknown" severity. Clearly we cannot use accidents whose severity is unknown to train/test a model designed to predict accident severity, so these rows will be dropped.

Columns containing useless/redundant data. These columns can be removed from the dataframe and include:

OBJECTID - this is just an database key, but Pandas creates its own key on import.
COLDETKEY - this is just a duplicate of INCKEY, and probably arises due to a cross-matching of tables on the Seattle Open Data website. We will keep INCKEY as a unique identifier for each accident, but do not need to keep the duplicate column
REPORTNO - this is just another identifier for the accident, this time tying the record to the individual piece of paperwork that was filed to report the accident. This is not useful for building our model.
STATUS - the meaning of this column is unclear (and is not explained in the Attribute Information metadata). The values are either "Matched" or "Unmatched".
EXCEPTRSNCODE, EXCEPTRSNDESC - these columns are listed in the Attribute Information metadata, but their meanings are not explained. EXCEPTRSNCODE is blank/NaN in ~99% of the data, with 2,480 accidents having a non-blank entry, all of which are the same ("NEI"). According to EXCEPTRSNDESC this means "Not Enough Information".
INCDATE - this column is just a duplicate of the more easily parsable INCDTTM
SDOTCOLNUM - this is another unique identifier for each accident, however as we are planning to keep INCKEY, keeping this second unique identifier is redundant.
Rows which are missing information about some of the features which we expect will be key to building the model: as we can see from df.head, a number of accidents (accounting for ~15% of the dataset) have "Unknown" values for attributes like WEATHER , ROADCOND and LIGHTCOND , or have these fields blank/NaN, which in practice means the same thing. As these are expected to be among the features which influence the likelihood and severity of accidents, we have to consider discarding these rows before training the model.

Presence of features with categorical values: In order to construct a model using Machine Learning techniques, we must take columns which contain categorical data and re-cast them in numeric form. Different techniques will be used to accomplish this, depending on the nature of the data in these columns.

Awkwardly, the target variable SEVERITYCODE is itself a categorical variable, having the values [0, 1, 2, 2b, 3]. For reasons given previously, we will be dropping all accidents with SEVERITYCODE =0, however we are still left with the categorical value '2b'. To facilitate modelling, I will switch the codes [0, 1, 2, 2b, 3] for codes [0, 1, 2, 3, 4], i.e. relabelling '2b' as '3', and relabelling '3' as '4'.
Some columns, such as UNDERINFL and HITPARKEDCAR contain a mixture of alphanumeric (Y/N or 1/0), boolean (True/False) and missing (NaN) data. To prepare the data for modelling it will be important to homogenise these data by treating 1, Y and True as equivalent (and setting these to 1) and treating 0, N and False as equivalent (setting these to 0). Furthermore, we can infer that missing values (represented with NaN) are equivalent to 0/False.
Other columns contain labelled data (e.g. WEATHERCOND , which takes one of a handful of values such as RAIN , CLEAR , SNOWING , etc). These can be prepared for modelling by using One-Hot Encoding, wherein a new column is created for each of the discrete values corresponding to the original variable/column, which is filled with 1s or 0s depending on the value in that column.
Incorporating timestamp information: The INCDTTM column contains the incident date/time in alphanumeric form. This can be parsed to an actual timestamp using Pandas, and then separated in to separate columns for Hour, Day (of month) Day (of week), Month and Year to study temporal trends

Other issues: after performing the above data engineering issues, we will create a copy of the data frame (minus any columns which will not be used for model-building) and perform a final check for any remaining NaNs/missing data. We will decide how to handle these later.

Okay, let's get started!

In [14]:
#Check for and remove any rows with SEVERITYDESC = "Unknown"
print("Give a run-down of the values present in the table for SEVERITYCODE:")
print(df["SEVERITYDESC"].value_counts())

#Identify and remove rows with SEVERITYDESC = "Unknown"
todrop = df["SEVERITYDESC"] == 'Unknown'
print("\n Preparing to drop "+str(todrop.values.sum())+" rows.")
df.drop(df.index[todrop], inplace=True)
print("Done!")

#Re-index dataframe to account for missing rows
df.reset_index(inplace=True)

Give a run-down of the values present in the table for SEVERITYCODE:
Property Damage Only Collision    136485
Injury Collision                   58188
Name: SEVERITYDESC, dtype: int64

 Preparing to drop 0 rows.
Done!


In [ ]:
#Delete unused columns
if 'OBJECTID' in df:
    del df["OBJECTID"]
if 'COLDETKEY' in df:
    del df["COLDETKEY"]
if 'REPORTNO' in df:
    del df["REPORTNO"]
if 'STATUS' in df:
    del df["STATUS"]
if 'EXCEPTRSNCODE' in df:
    del df["EXCEPTRSNCODE"]
if 'EXCEPTRSNDESC' in df:
    del df["EXCEPTRSNDESC"]
if 'INCDATE' in df:
    del df["INCDATE"]
if 'SDOTCOLNUM' in df:
    del df["SDOTCOLNUM"]
if 'STCOLCODE' in df:
    del df["STCOLCODE"]

In [ ]:
#Find missing WEATHERCOND
todrop1 = df["WEATHER"] == 'Unknown'
noweatherinfo = todrop1.values.sum()

#Find missing ROADCOND
todrop2 = df["ROADCOND"] == "Unknown"
noroadcondinfo = todrop2.values.sum()

#Find missing LIGHTCOND
todrop3 = df["LIGHTCOND"] == "Unknown"
nolightinfo = todrop3.values.sum()

#Collate these and remove
df["TODROP"] = 0
count_noweather = 0
count_noinfo = 0
for i in range(0,len(todrop1)):
    if todrop1[i] == True or todrop2[i] == True or todrop3[i] == True:
        df["TODROP"][i] = 1
        
print("There are "+str(noweatherinfo)+" accidents with no weather information.")
print("There are "+str(noroadcondinfo)+" accidents with no road condition information.")
print("There are "+str(nolightinfo)+" accidents with no information about light conditions.")
print("There are "+str(df["TODROP"].values.sum())+" accidents without one or more of the above.\n Deleting now...")

#Delete the temporary column "TODROP" and re-index the data
shape0 = df.shape
todrop = df["TODROP"] == 1
df.drop(df.index[todrop], inplace=True)

#Print info about DF shape before and after
print("Shape of DF before: "+str(shape0[0])+" x "+str(shape0[1]))
print("Shape of DF after: "+str(df.shape[0])+" x "+str(df.shape[1]))
print("The original DF had shape: "+str(df_shape_orig[0])+" x "+str(df_shape_orig[1]))

#Re-index dataframe to account for missing rows
df.reset_index(inplace=True)

#Delete the temporary "TODROP" column from the DataFrame
if 'TODROP' in df:
    del df["TODROP"]
    del df["level_0"]
    del df["index"]

In [ ]:
#Make the target variable numerical by changing [0, 1, 2, 2b, 3] to [0, 1, 2, 3, 4]
#Print the range of codes before cleaning:
print(df["SEVERITYCODE"].value_counts())

#Clean...
count2b = 0
count3  = 0
for i in range(0,len(df["SEVERITYCODE"])):
    if df["SEVERITYDESC"][i] == 'Serious Injury Collision':
        df["SEVERITYCODE"][i] = 3
        count2b += 1
    if df["SEVERITYDESC"][i] == 'Fatality Collision':
        df["SEVERITYCODE"][i] = 4
        count3 += 1
        
#Make sure that SEVERITYCODE is cast as an integer, rather than an object
df = df.astype({'SEVERITYCODE':np.int})
        
#Confirm that the range of codes after cleaning is as expected:
print("\n Changed "+str(count2b)+" entries from 2b->3 and "+str(count3)+" from 3->4. \n The tables now look like:")
print(df["SEVERITYCODE"].value_counts())
print(df["SEVERITYDESC"].value_counts())

In [ ]:
#Take columns with mixed boolean data types ([1, "Y", True],[0, "N", False] etc) and cast them as numerical variables
#SPEEDING, INATTENTIONIND, UNDERINFL, PEDROWNOTGRNT, HITPARKEDCAR
df["SPEEDING"].replace(np.nan, 0, inplace=True)
df["SPEEDING"].replace("Y", 1, inplace=True)

df["INATTENTIONIND"].replace(np.nan, 0, inplace=True)
df["INATTENTIONIND"].replace("Y", 1, inplace=True)

df["UNDERINFL"].replace(np.nan, 0, inplace=True)
df["UNDERINFL"].replace('N', 0, inplace=True)
df["UNDERINFL"].replace('0', 0, inplace=True)
df["UNDERINFL"].replace('1', 1, inplace=True)
df["UNDERINFL"].replace("Y", 1, inplace=True)

df["PEDROWNOTGRNT"].replace(np.nan, 0, inplace=True)
df["PEDROWNOTGRNT"].replace("Y", 1, inplace=True)

df["HITPARKEDCAR"].replace("N", 0, inplace=True)
df["HITPARKEDCAR"].replace(np.nan, 0, inplace=True)
df["HITPARKEDCAR"].replace("Y", 1, inplace=True)

In [ ]:
df["WEATHER"].value_counts()

In [ ]:
df["LIGHTCOND"].value_counts()

In [ ]:
df["ROADCOND"].value_counts()

In [ ]:
df["ADDRTYPE"].value_counts()

In [ ]:
#Use One Hot Encoding to deal with categorical data
#Get dummies for one-hot encoding
weatherdummies = pd.get_dummies(df["WEATHER"])
lightconddummies = pd.get_dummies(df["LIGHTCOND"])
roadconddummies = pd.get_dummies(df["ROADCOND"])
addrtypedummies = pd.get_dummies(df["ADDRTYPE"])

#There is an "other" column in each of the above DFs -- rename these to avoid confusion
#when the DataFrames are merged
weatherdummies.rename(columns={"Other": "Other weather"}, inplace=True)
lightconddummies.rename(columns={"Other": "Other light"}, inplace=True)
roadconddummies.rename(columns={"Other": "Other roadcond"}, inplace=True)

#Now use one-hot encoding on SDOT_COLCODE
colcodedummies = pd.get_dummies(df["SDOT_COLCODE"])
columns = colcodedummies.columns
for col in columns:
    colcodedummies = colcodedummies.rename(columns={col: 'SDOT_COLCODE_'+str(col)})

#Join columns to existing DataFrame
#data_frames = [df, weatherdummies, lightconddummies, roadconddummies, addrtypedummies]
df = df.join(weatherdummies)
df = df.join(lightconddummies)
df = df.join(roadconddummies)
df = df.join(addrtypedummies)
df = df.join(colcodedummies)

In [ ]:
print(colcodedummies.shape)
colcodedummies.head(5)

In [ ]:
#Separate Date/Time from INCDTTM colum and create columns for Hour/Minute, Day/Month/Year
df["INC_DATE_TIME"] = pd.to_datetime(df["INCDTTM"])
df["MONTH"] = pd.DatetimeIndex(df["INC_DATE_TIME"]).month
df["DAY_OF_WEEK"] = df["INC_DATE_TIME"].dt.dayofweek
df["DAY_OF_MONTH"] = pd.DatetimeIndex(df["INC_DATE_TIME"]).day
df["YEAR"] = pd.DatetimeIndex(df["INC_DATE_TIME"]).year

#Find the nearest whole hour to the time the collision occurred, e.g. 14:25 -> 14:00 and 14:35 -> 15:00
from datetime import datetime, timedelta

#Create new columns
df["TIME_ROUNDED_TO_NEAREST_HOUR"] = 0
df["DAY_STRING"] = 0
df["MONTH_STRING"] = 0
df["WEEKEND"] = 0

#Define some useful functions
def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
               +timedelta(hours=t.minute//30))

def get_dow(day_int):
    if day_int == 0:
        dow = 'Mon'
    if day_int == 1:
        dow = 'Tue'
    if day_int == 2:
        dow = 'Wed'
    if day_int == 3:
        dow = 'Thu'
    if day_int == 4:
        dow = 'Fri'
    if day_int == 5:
        dow = 'Sat'
    if day_int == 6:
        dow = 'Sun'
    return dow

def get_month(month_int):
    if month_int == 1:
        monthstring = 'Jan'
    if month_int == 2:
        monthstring = 'Feb'
    if month_int == 3:
        monthstring = 'Mar'
    if month_int == 4:
        monthstring = 'Apr'
    if month_int == 5:
        monthstring = 'May'
    if month_int == 6:
        monthstring = 'Jun'
    if month_int == 7:
        monthstring = 'Jul'
    if month_int == 8:
        monthstring = 'Aug'
    if month_int == 9:
        monthstring = 'Sep'
    if month_int == 10:
        monthstring = 'Oct'
    if month_int == 11:
        monthstring = 'Nov'
    if month_int == 12:
        monthstring = 'Dec'
    return monthstring
     
def is_weekend(day_int):
    if day_int <= 4:
        isweekend = 0
    else:
        isweekend = 1
    return isweekend

start_time = time.time()
#Get the day of the week from its integer value (0-6)
print("Get the day of the week from its integer value (0-6):")
with concurrent.futures.ProcessPoolExecutor(num_processes) as pool:
    df["DAY_STRING"] = list(tqdm(pool.map(get_dow, df["DAY_OF_WEEK"], chunksize=10), total=df.shape[0]))
    
#Get the name of the month from its integer value (1-12)
print("Get the name of the month from its integer value (1-11):")
with concurrent.futures.ProcessPoolExecutor(num_processes) as pool:
    df["MONTH_STRING"] = list(tqdm(pool.map(get_month, df["MONTH"], chunksize=10), total=df.shape[0]))
    
#Round time to the nearest whole hour
print("Round time to the nearest whole hour:")
with concurrent.futures.ProcessPoolExecutor(num_processes) as pool:
    df["TIME_ROUNDED_TO_NEAREST_HOUR"] = list(tqdm(pool.map(hour_rounder, df["INC_DATE_TIME"], chunksize=10), total=df.shape[0]))
    
#Check if an accident occurred on the weekend or not
print("Check if an accident occurred on the weekend or not:")
with concurrent.futures.ProcessPoolExecutor(num_processes) as pool:
    df["WEEKEND"] = list(tqdm(pool.map(is_weekend, df["DAY_OF_WEEK"], chunksize=10), total=df.shape[0]))
       
elapsed_time = time.time() - start_time
print("Elapsed time for parallel FOR loop (seconds): "+str(elapsed_time))
        
#Now store the nearest hour to the time of the accident as an integer and delete the rounded Datetime
df["HOUR_NEAREST"] = pd.DatetimeIndex(df["TIME_ROUNDED_TO_NEAREST_HOUR"]).hour
del df["TIME_ROUNDED_TO_NEAREST_HOUR"]

#Check the result of this
df.head(5)

In [ ]:
#Check the dataframe for any NaNs
print(df.isnull().sum(axis=0))

In [ ]:
#ST_COLCODE
#Find NaNs and set these to spaces.
print("How many are null: "+str(df["ST_COLCODE"].isnull().sum()))
print("Try to populate these with -999")
df["ST_COLCODE"].fillna(' ', inplace=True)
print("How many are null: "+str(df["ST_COLCODE"].isnull().sum()))

#Get rid of accidents with no ST_COLCODE (null and/or former NaNs)
todrop = df["ST_COLCODE"] == ' '

#Collate these and remove
df["TODROP"] = 0
for i in range(0,len(todrop)):
    if todrop[i] == True:
        df["TODROP"][i] = 1
        
print("There are "+str(df["TODROP"].values.sum())+" accidents without an ST_COLCODE.\n Deleting now...")

#Delete the temporary column "TODROP" and re-index the data
todrop = df["TODROP"] == 1
df.drop(df.index[todrop], inplace=True)

#Re-index dataframe to account for missing rows
df.reset_index(inplace=True)

#Delete the temporary "TODROP" column from the DataFrame
if 'TODROP' in df:
    del df["TODROP"]
    del df["index"]

#Cast from type 'object' to type 'int'
df = df.astype({'ST_COLCODE':np.int64})
    
print("Dataframe shape now:")
df.shape

In [ ]:
#Missing geographical data
#Find null values and replace these with NaN
#Find NaNs and set these to spaces.
print("There are "+str(df["X"].isnull().sum())+" null location values. Will try to get X, Y data for these based on the locations of other accidents that share the same INTKEY.")

#Go through the accidents with no X,Y data and see if we can match them to others
#that share the same INTKEY
df["NOT_HAS_COORDS"] = df["X"].isnull()
count = 0
for i in range(0,len(df["X"])):
    if df["NOT_HAS_COORDS"][i] == True:
        if df["INTKEY"][i] > 0:
            #Find other accidents which share the same INTKEY
            matched_intersections = df.loc[df["INTKEY"] == df["INTKEY"][i]]
            #Do any of these have the required X,Y data?
            with_geodata = matched_intersections["X"].count()
            if with_geodata > 0:
                matched_intersections.head()
                count += 1

if "NOT_HAS_COORDS" in df:
    del df["NOT_HAS_COORDS"]
    
print("Added locations for "+str(count)+" accidents based on shared INTKEYS.")

In [ ]:
#Remove accidents from the database which still have no location information
#X, Y
#Find NaNs and set these to spaces.
print("There are "+str(df["X"].isnull().sum())+" accidents with no location information")
print("Populate these with -999 and then delete")
df["X"].fillna(-999, inplace=True)
print("There are now "+str(df["ST_COLCODE"].isnull().sum())+" with NaN/null locations.")
print("I will now delete these...")

#Get rid of accidents with no ST_COLCODE (null and/or former NaNs)
todrop = df["X"] == -999

#Collate these and remove
df["TODROP"] = 0
for i in range(0,len(todrop)):
    if todrop[i] == True:
        df["TODROP"][i] = 1
        
print("There are "+str(df["TODROP"].values.sum())+" accidents without an location.\n Deleting now...")

#Delete the temporary column "TODROP" and re-index the data
todrop = df["TODROP"] == 1
df.drop(df.index[todrop], inplace=True)

#Re-index dataframe to account for missing rows
df.reset_index(inplace=True)

#Delete the temporary "TODROP" column from the DataFrame
if 'TODROP' in df:
    del df["TODROP"]
    del df["index"]

print("Dataframe shape now:")
df.shape

In [ ]:
#Keep the full dataframe, but create a copy which drops columns which will not be used for modelling
data = df.copy()
if 'INCKEY' in df:
    del data["INCKEY"]
    del data["INTKEY"]
    del data["ADDRTYPE"]
    del data["LOCATION"]
    del data["SEVERITYDESC"]
    del data["COLLISIONTYPE"]
    del data["INCDTTM"]
    del data["JUNCTIONTYPE"]
    del data["SDOT_COLDESC"]

In [ ]:
print("The shape of the dataframe is:")
print(data.shape)

In [ ]:
print("The data types are:")
data.dtypes

In [ ]:
print("Here is a count of how many null/NaN values there are in each column:")
print(data.isnull().sum(axis=0))

# Data Exploration 1: Where do accidents occur?
The Seattle Open Data portal record of traffic accidents include the latitude and longitude (as X, Y, respectively) of every accident that occurred in the city council area. Let's create a map in Folium to see where these occur. This might highlight some key "choke points" in the city road network and give context for some of the predictions that eventually come out of the model.

In [ ]:
import folium

#Get the median latitude and logitude and centre the map there
lon_med = data["X"].mean()
lat_med = data["Y"].mean()

print(lat_med)
print(lon_med)

seattle_map = folium.Map(location=[lat_med, lon_med], zoom_start=11)
seattle_map

In [ ]:
import pyscreenshot as ImageGrab
from selenium import webdriver

#im=ImageGrab.grab()
#im.save('map1_seattle.png', format='png')
##im=ImageGrab.grab(bbox=(500,250,1480,830))

#Read the Seattle geojson data
import json
import requests

#GeoJSON file obtained from https://raw.githubusercontent.com/seattleio/seattle-boundaries-data/master/data/neighborhoods.geojson
#!wget https://raw.githubusercontent.com/seattleio/seattle-boundaries-data/master/data/neighborhoods.geojson
geofile = 'neighborhoods.geojson'

with open(geofile) as json_file:
    geodata = json.load(json_file)

print("There are "+str(len(geodata['features']))+" districts in the Seattle area:")

In [ ]:
#Convert latitude/longitude to point objects and find out which area they lie in
from shapely.geometry import shape, Point, Polygon
from shapely.ops import nearest_points
from time import strftime, gmtime
import random

    
def find_county(lon, lat):
    point = Point(lon, lat)
    found_nhood = 0
    '''Most accidents can be trivially located within a neighbourhood
       using the polygon.contains() function from shapely
    '''
    for i in range(0,len(geodata['features'])):
        polygon = shape(geodata['features'][i]['geometry'])
    if polygon.contains(point):
            nhood = geodata['features'][i]['properties']['name']
            found_nhood = 1
    if found_nhood == 0:
        '''Some accidents can't be located within neighbourhoods because
           they occur on major highways which mark neighbourhood boundaries:
           the "no man's land" problem. Let's deal with these by assigning
           them to the _nearest_ neighbourhood
        '''
        nhoods = []
        distances = np.zeros(len(geodata['features']))
        for i in range(0,len(geodata['features'])):
            polygon = shape(geodata['features'][i]['geometry'])
            distances[i] = point.distance(polygon)
            nhoods.append(geodata['features'][i]['properties']['name'])
            
        min_distance = np.where(distances == np.amin(distances)) 
        try:
            #Usually this finds the nearest district
            min_distance_index = int(min_distance[0])
        except:
            '''But sometimes the accident is exactly equidistant between
               two neighbourhood boundaries. In this case, choose one
               of the districts at random to avoid biasing the sample
            '''
            min_distance_index = int(random.choice(min_distance[0]))
        nhood = nhoods[min_distance_index]
        found_nhood = 1
        if found_nhood == 0:
            sys.exit("I'm sorry I can't do that Dave...")
    return nhood
    
start_time = time.time()

from tqdm.auto import tqdm, trange
with concurrent.futures.ProcessPoolExecutor(num_processes) as pool:
    data["NEIGHBOURHOOD"] = list(tqdm(pool.map(find_county, data['X'], data['Y'], chunksize=10), total=df.shape[0]))
    
        
elapsed_time = time.time() - start_time
print("Multi-thread elapsed time: "+str(strftime("%H:%M:%S", gmtime(elapsed_time))))

In [ ]:
print("There are "+str(data["NEIGHBOURHOOD"].isnull().sum())+" accidents with no neighbourhood information")
print("The number of accidents in each neighbourhood is as follows:")
data["NEIGHBOURHOOD"].value_counts()

In [ ]:
#Now count how many accidents there are in each neighbourhood
accidents_per_nhood = pd.DataFrame(data["NEIGHBOURHOOD"].value_counts())
accidents_per_nhood.reset_index(inplace=True)
accidents_per_nhood.rename(columns = {'NEIGHBOURHOOD':"COUNT"}, inplace=True)
accidents_per_nhood.rename(columns = {'index':"NEIGHBOURHOOD"}, inplace=True)

#Also calculate the average severity of accidents in each area for later
temp_df = data.groupby(['NEIGHBOURHOOD']).mean()
temp_df.reset_index(inplace=True)
accidents_per_nhood["MEAN_SEVERITY"] = 0.0
for i in range(0,len(accidents_per_nhood["NEIGHBOURHOOD"])):
    for j in range(0,len(temp_df["X"])):
        if accidents_per_nhood["NEIGHBOURHOOD"][i] == temp_df["NEIGHBOURHOOD"][j]:
            accidents_per_nhood["MEAN_SEVERITY"][i] = temp_df["SEVERITYCODE"][j]

accidents_per_nhood.head(5)

In [ ]:
print(folium.__version__)

#Now use the geojson data and create a choropleth map
threshold_scale = np.linspace(accidents_per_nhood['COUNT'].min(),
                              accidents_per_nhood['COUNT'].max(),
                              6, dtype=int)
threshold_scale = threshold_scale.tolist() # change the numpy array to a list
threshold_scale[-1] = threshold_scale[-1] + 1 

# generate choropleth map using the accident data
seattle_map = folium.Map(location=[lat_med, lon_med], zoom_start=11)
seattle_map.choropleth(
    geo_data=geodata,
    data=accidents_per_nhood,
    columns=['NEIGHBOURHOOD', 'COUNT'],
    key_on='feature.properties.name',
    threshold_scale=threshold_scale,
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    nan_fill_opacity=0.0,
    nan_line_opacity=0.0,
    legend_name='N accidents per neighbourhood',
    reset=True
)

# display map
seattle_map

In [ ]:
#im=ImageGrab.grab()
#im.save('map2_number_of_accidents.png', format='png')
#im=ImageGrab.grab(bbox=(500,250,1480,830))

In [ ]:
#Accident severity map
threshold_scale = np.linspace(accidents_per_nhood['MEAN_SEVERITY'].min(),
                              accidents_per_nhood['MEAN_SEVERITY'].max(),
                              6, dtype=int)
threshold_scale = threshold_scale.tolist() # change the numpy array to a list
threshold_scale[-1] = threshold_scale[-1] + 1 

#Generate a choropleth map of the mean accident severity in each neighbourhood
seattle_map = folium.Map(location=[lat_med, lon_med], zoom_start=11)
seattle_map.choropleth(
    geo_data=geodata,
    data=accidents_per_nhood,
    columns=['NEIGHBOURHOOD', 'MEAN_SEVERITY'],
    key_on='feature.properties.name',
    #threshold_scale=threshold_scale,
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    nan_fill_opacity=0.0,
    nan_line_opacity=0.0,
    legend_name='Average severity code of accidents',
    reset=True
)

# display map
seattle_map

In [ ]:
#im=ImageGrab.grab()
#im.save('map3_accident_severity.png', format='png')
#im=ImageGrab.grab(bbox=(500,250,1480,830))

In [ ]:
#Import Matplotlib and prepare inline plotting
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from collections import Counter

#Set up gpplot style
mpl.style.use('ggplot')

# check for latest version of Matplotlib
print ('Matplotlib version: ', mpl.__version__) # >= 2.0.0
df["ADDRTYPE"].value_counts()

In [ ]:

print("There are "+str(len(df))+" entries in df currently.")

#Create a 2x2 set of bar charts to describe the accident data
plt.rcParams["figure.figsize"] = (16,16)
#1 - Accident severities
plt.subplot(2,2,1)
freqs = Counter(df["SEVERITYDESC"])
xvals = range(len(freqs.values()))
plt.title("Distribution of accident outcomes")
plt.title('(a)')
plt.ylabel("Number of accidents")
plt.grid(b=None)
plt.bar(xvals, freqs.values() , color='#37777D')
plt.xticks(xvals, freqs.keys(), rotation='vertical')
plt.yscale('log')

#2 - Number of people involved
plt.subplot(2,2,2)
freqs = Counter(df["PERSONCOUNT"])
xvals = range(len(freqs.values()))
plt.title('(b)')
plt.xlabel("Number of people involved")
plt.ylabel("log(Number of accidents)")
plt.grid(b=None)
plt.hist(df["PERSONCOUNT"], align='left', color='#37777D')
plt.yscale('log')

#3 - Number of people involved
plt.subplot(2,2,3)
freqs = Counter(df["INJURIES"])
xvals = range(len(freqs.values()))
plt.title('(c)')
plt.xlabel("Number of injuries")
plt.ylabel("log(Number of accidents)")
plt.grid(b=None)
plt.hist(df["INJURIES"], align='left', color='#37777D')
plt.yscale('log')

#4 - Number of fatalities per accident
plt.subplot(2,2,4)
freqs = Counter(df["FATALITIES"])
xvals = range(len(freqs.values()))
plt.title('(d)')
plt.xlabel("Number of fatalities")
plt.ylabel("log(Number of accidents)")
plt.grid(b=None)
plt.hist(df["FATALITIES"], bins=[0,1,2,3,4,5,6], align='left', color='#37777D')
plt.yscale('log')

plt.subplots_adjust(hspace=0.6)
#plt.savefig('./FigZZZ_accident_histograms.png', bbox_inches='tight')

In [ ]:
#Metadata for the charts that follow:
#1 - Weather conditions
print('Frequency of weather types:')
print(df["WEATHER"].value_counts())
print(len(df["WEATHER"]))

In [ ]:
#2 - Road conditions
print('Frequency of rifferent road conditions:')
print(df["ROADCOND"].value_counts())

In [ ]:
#3 - Light conditions
print('Frequency of different light conditions:')
print(df["LIGHTCOND"].value_counts())

In [ ]:
#4 - Month of year in which accident happens
print('Months of the year (there should be 12...)')
print(df["MONTH"].value_counts())

In [ ]:
#Create a 2x2 set of bar charts to describe the accident data
plt.rcParams["figure.figsize"] = (16,16)
#1 - Weather conditions
plt.subplot(2,2,1)
freqs = Counter(df["WEATHER"])
xvals = range(len(freqs.values()))
plt.title('(a)')
plt.ylabel("Number of accidents")
plt.grid(b=None)
plt.bar(xvals, freqs.values() , color='#37777D')
plt.xticks(xvals, freqs.keys(), rotation='vertical')

#2 - Road conditions
plt.subplot(2,2,2)
freqs = Counter(df["ROADCOND"])
xvals = range(len(freqs.values()))
plt.title('(b)')
plt.ylabel("Number of accidents")
plt.grid(b=None)
plt.bar(xvals, freqs.values() , color='#37777D')
plt.xticks(xvals, freqs.keys(), rotation='vertical')

#3 - Light conditions
plt.subplot(2,2,3)
freqs = Counter(df["LIGHTCOND"])
xvals = range(len(freqs.values()))
plt.title('(c)')
plt.ylabel("Number of accidents")
plt.grid(b=None)
plt.bar(xvals, freqs.values() , color='#37777D')
plt.xticks(xvals, freqs.keys(), rotation='vertical')

#4 - ADDRTYPE
plt.subplot(2,2,4)
freqs = Counter(df["COLLISIONTYPE"])
xvals = range(len(freqs.values()))
plt.title('(d)')
plt.ylabel("Number of accidents")
plt.grid(b=None)
plt.bar(xvals, freqs.values() , color='#37777D')
plt.xticks(xvals, freqs.keys(), rotation='vertical')
plt.subplots_adjust(hspace=0.6)
#plt.savefig('./FigZZZ_accidents_by_roadcond.png', bbox_inches='tight')

In [ ]:
#Create a 2x2 set of bar charts to describe the accident data
plt.rcParams["figure.figsize"] = (16,8)

#1 - Day of week in which accident happens
plt.subplot(1,2,1)
freqs = Counter(df["DAY_OF_WEEK"])
xvals = range(len(freqs.values()))
#Order the days
ordered = dict(sorted(freqs.items()))
daynames = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
plt.title('(a)')
plt.xlabel("Day of week")
plt.ylabel("Number of accidents")
plt.grid(b=None)
plt.bar(xvals, ordered.values() , color='#37777D')
plt.xticks(xvals, daynames, rotation='vertical')

#2 - Month of year in which accident happens
plt.subplot(1,2,2)
freqs = Counter(df["MONTH"])
xvals = range(len(freqs.values()))
#Order the months
ordered = dict(sorted(freqs.items()))
monthnames = ['Jan','Feb','Mar','Apr','May', 'Jun','Jul','Aug','Sep','Oct','Nov','Dec']
plt.title('(b)')
plt.xlabel("Month")
plt.ylabel("Number of accidents")
plt.grid(b=None)
plt.bar(xvals, ordered.values() , color='#37777D')
plt.xticks(xvals, monthnames, rotation='vertical')
plt.subplots_adjust(hspace=0.6)
#plt.savefig('./FigZZZ_accidents_by_DOW_month.png', bbox_inches='tight')

In [ ]:
#Count how many accidents occurred on each month since the beginning of the dataset

#Plot showing the number of accidents in each month since time began
#Set a monthrange between the first and last months in the dataframe:
monthssorted = sorted(df["INC_DATE_TIME"])
month_range = pd.period_range(start=monthssorted[0],end=monthssorted[-1],freq='M')
month_df = pd.DataFrame(month_range, columns=["YEAR_MONTH"])

#Index by time
new_df = pd.DataFrame(sorted(df["INC_DATE_TIME"]),columns=['INC_DATE_TIME'])
new_df["DATETIME"] = 0

def todatetime(time):
    output = time.to_pydatetime()
    return output.date()

from tqdm.auto import tqdm, trange
with concurrent.futures.ProcessPoolExecutor(num_processes) as pool:
    new_df["DATETIME"] = list(tqdm(pool.map(todatetime, new_df["INC_DATE_TIME"], chunksize=10), total=new_df.shape[0]))
new_df.set_index("DATETIME", inplace=True)

#Iterate over months and sum up the number of accidents in each
accidents_this_month = [0]*len(month_range)
accidents_this_month_daily = [0]*len(month_range)
days_in_month = np.zeros(len(month_range))
for i in range(0,len(month_range)):
    startdate = month_range[i].start_time.date()#.to_pydatetime()
    enddate = month_range[i].end_time.date()#.to_pydatetime()
    #Loop over the dataframe
    days_in_month[i] = month_range[i].days_in_month
    accidents_this_month[i] = len(new_df.loc[startdate:enddate]) 
    accidents_this_month_daily[i] = len(new_df.loc[startdate:enddate]) / days_in_month[i]
    
month_df["ACCIDENTS_THIS_MONTH"] = accidents_this_month
month_df["ACCIDENTS_THIS_MONTH_DAILY"] = accidents_this_month_daily
month_df["DAYS_IN_MONTH"] = days_in_month
month_df.head(15)

In [ ]:
import matplotlib.dates as mdates

#Now plot the number of accidents for each month in the dataset
plt.rcParams["figure.figsize"] = (16,8)

#Convert Period to a fixed date for the start of each month
month_df["START_OF_MONTH"] = [0] * len(month_df)
for i in range(0,len(month_df)):
    month_df["START_OF_MONTH"][i] = month_df["YEAR_MONTH"][i].start_time.date()
    
#Get a three-month rolling average
month_df["SIX_MONTH_AVG"] = month_df.iloc[:,1].rolling(window=6).mean()

ax = plt.gca()
plt.rc('xtick',labelsize=10)
plt.rc('ytick',labelsize=10)
years = mdates.YearLocator()   
months = mdates.MonthLocator()
ax.xaxis.set_major_locator(years)
ax.xaxis.set_minor_locator(months)
plt.xlabel("Year", size=28)
plt.ylabel("Number of accidents per month", size=28)
plt.plot(month_df["START_OF_MONTH"], month_df["SIX_MONTH_AVG"], color='#37777D')
plt.scatter(month_df["START_OF_MONTH"], month_df["ACCIDENTS_THIS_MONTH"], color='#37777D')
#plt.savefig('./Figs/Fig1_accidents_per_month.png', bbox_inches='tight')

In [ ]:
import matplotlib.dates as mdates

#Now plot the number of *daily* accidents for each month in the dataset
plt.rcParams["figure.figsize"] = (16,8)

#Get a three-month rolling average
month_df["SIX_MONTH_AVG_DAILY"] = month_df.iloc[:,2].rolling(window=6).mean()

plt.rc('xtick',labelsize=11)
plt.rc('ytick',labelsize=10)

ax = plt.gca()
years = mdates.YearLocator()   
months = mdates.MonthLocator()
ax.xaxis.set_major_locator(years)
ax.xaxis.set_minor_locator(months)
plt.xlabel("Date", size=24)
plt.ylabel("Number of accidents per day", size=24)
plt.plot(month_df["START_OF_MONTH"], month_df["SIX_MONTH_AVG_DAILY"], color='#37777D')
plt.scatter(month_df["START_OF_MONTH"], month_df["ACCIDENTS_THIS_MONTH_DAILY"], color='#37777D')
plt.savefig('./Fig5_daily_accidents_per_month.png', bbox_inches='tight')

# Pre-Processing: Feature extraction
We now enter the final phase of data preparation -- deciding which of the remaining columns from the dataframe to include in our model building. Let's begin by looking at a correlation matrix for the dataset

In [ ]:
if "Alley" in data:
    del data["Alley"]

plt.rcParams["figure.figsize"] = (18,16)
corr = data.corr()
plt.rc('xtick',labelsize=10)
plt.rc('ytick',labelsize=10)
sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns)
plt.savefig('./Figs/Fig3_correlation_unscaled.png', bbox_inches='tight')